In [1]:
import numpy as np
import pylab as plt
from numpy import linalg as la
%matplotlib inline

Let $(X,U,P,L)$ be an MDP, i.e.

* $X$ is a finite set of $n$ *states*
* $U$ is a finite set of $m$ *actions*
* $P:X\times U\rightarrow \Delta(X)$ is transition probability
* $\mathcal{L}:X\times U\times X\rightarrow \mathbb{R}$ is the 1-stage cost (for reward, use $-\mathcal{L}$)

where $\Delta(S) = \{p\in[0,1]^S : \sum_{s\in S} p(s) = 1\}$ is the set of probability distributions over the finite set $S$.

In [17]:
n,m = 5,2
X = range(n)
U = range(m)
# generate transition probabilities
P = np.abs(np.random.randn(n,m,n))
P = P / P.sum(axis=2)[...,np.newaxis]
assert np.all(P.min(axis=2) > 0),'no probability is zero'
assert np.all(P.max(axis=2) < 1),'no probability is one'
assert np.allclose(P.sum(axis=2), 1.),'probabilities sum to 1'
# generate rewards
L = np.abs(np.random.randn(n,m,n))

Starting with an initial state distribution $p\in \Delta(X)$ and policy $\pi:X\rightarrow \Delta(U)$, can simulate by iterating:
$$p(x)^+ = \sum_{\xi\in X} p(\xi) \sum_{\mu\in U} \pi(\xi)(\mu) P(\xi,\mu)(x).$$
Equivalently, defining $\Gamma : X \rightarrow \Delta(X)$ by
$$\forall \xi,x\in X : \Gamma(\xi)(x) = \sum_{\mu\in U} \pi(\xi)(\mu) P(\xi,\mu)(x)$$
and treating $p$ as a row vector, we can iterate $p^+ = p \Gamma$.

In [18]:
# initial state distribution
p0 = np.abs(np.random.randn(n)); p0 = p0 / np.sum(p0);
assert np.allclose(np.sum(p0),1.)
# policy
pi = np.abs(np.random.randn(n,m)); pi = pi / np.sum(pi,axis=1)[:,np.newaxis]
assert np.allclose(np.sum(pi,axis=1),1.)

In [19]:
# iterative simulation
def sim_p(X,U,P,R,p0,pi,T=1):
    n = len(X); m = len(U)
    p_ = [p0]
    for t in range(T):
        p  = p_[-1].copy()
        _p = np.nan*p
        for x in X:
            _p[x] = np.sum([[p[xi] * pi[xi,mu] * P[xi,mu,x] for mu in U] for xi in X])
        assert np.allclose(np.sum(_p),1.),"simulation must preserve probabilities"
        p_.append(_p)
    return p_

# matrix multiplication
#G = np.zeros((n,n))
#for xi in X:
#    for x in X:
#        G[xi,x] = np.sum([pi[xi,mu] * P[xi,mu,x] for mu in U])
# this can actually be done in one line
G = np.sum(pi[...,np.newaxis]*P,axis=1)
assert np.allclose(np.sum(G,axis=1),1.)


# sanity check:  matrix multiplication yields same result as iterative simulation
# (note:  p0 is a row vector, so we're doing left multiplication on G)
assert np.allclose(sim_p(X,U,P,R,p0,pi)[-1], np.dot(p0,G))

Since $\Gamma$ is left-stochastic (row sums equal to 1) and ergodic (every state reachable from every other under policy $\pi$), it has one unity eigenvalue, and all other eigenvalues are contractive (magnitude smaller than 1):

In [20]:
print('spec Gamma =',la.eigvals(G))
assert np.all(np.abs(la.eigvals(G)[1:]) < 1)

spec Gamma = [ 1.        +0.j          0.10544458+0.09731843j  0.10544458-0.09731843j
 -0.14669747+0.06941977j -0.14669747-0.06941977j]


This implies that any initial probability distribution will asymptotically converge to the eigenvector corresponding to the unity eigenvalue, which will have no zero entries:

In [21]:
v = la.eig(G.T)[1][:,0]; v /= v.sum()
assert np.allclose(v,np.dot(v,G)),'v is left-eigvec of Gamma with eigval 1'
assert np.allclose(v, np.dot(p0,la.matrix_power(G,100))),'p0 converges to v'
assert np.all(v > 0.),'all states have positive probability'

Let us now consider the problem of minimizing the infinite-horizon discounted cost
$$c(x,u) = \sum_{t=0}^\infty \gamma^t \mathcal{L}(x_t,u_t),$$
where $\gamma\in(0,1)$ is a *discount factor* and $\mathcal{L}(x_t,u_t)$ is the cost at time $t$.

In [22]:
g = np.random.rand()

Any policy $\pi : X\rightarrow\Sigma(U)$ has an associated *value* $v^\pi : X\rightarrow\mathbb{R}$ defined by
$$\forall \xi\in X : v^\pi(\xi) = E[c(x,u) \mid x_0 = \xi]$$
that satisfies the *Bellman equation*
$$\forall\xi\in X : v^\pi(\xi) = \sum_{\mu\in U}\pi(\mu \mid \xi)\sum_{x^+\in X} P(x^+\mid\xi,\mu)(\mathcal{L}(\xi,\mu) + \gamma\, v^\pi(x^+)).$$
Noting that $v^\pi$ appears linearly in the above equation, we can solve for $v^\pi$ given $\pi$ using linear algebra:
$$A_\pi v^\pi = b_\pi,$$
$$b_\pi(\xi) = \sum_{\mu\in U} \pi(\mu\mid\xi) \sum_{x^+\in X} P(x^+\mid\xi,\mu) \mathcal{L}(\xi,\mu,x),$$

$$A_\pi(\xi,x^+) = \delta(\xi,x^+) - \sum_{\mu\in U} \gamma\, \pi(\mu\mid\xi) P(x^+\mid\xi,\mu),$$
where $\delta:X\times X\rightarrow\{0,1\}$ is the *Kronecker delta*, i.e. $\delta(\xi,x) = 1 \iff \xi = x$.

In [26]:
def policy_evaluation(X,U,P,L,g,pi):
    b = np.asarray([ np.sum([[pi[xi,mu] * P[xi,mu,x] * L[xi,mu,x] 
                          for x in X] for mu in U]) for xi in X])
    I = np.identity(n)
    A = I - np.asarray([ np.asarray([ np.sum([g * pi[xi,mu] * P[xi,mu,x] 
                                              for mu in U]) for x in X]) for xi in X])
    # v satisfies Bellman equation
    v = np.dot(b,la.inv(A).T)
    assert np.allclose(v, [np.sum([[pi[xi,mu] * P[xi,mu,x] * (L[xi,mu,x] + g*v[x]) 
                                for x in X] for mu in U]) for xi in X])
    return v
print('value of policy:')
print(policy_evaluation(X,U,P,L,g,pi))

value of policy:
[19.84294402 19.6297296  20.36225594 19.83700439 19.74344903]


Given the value $v^\pi:X\rightarrow\mathbb{R}$ of any policy $\pi:X\rightarrow\Delta(U)$, we can perform *policy improvement* via:
$$\forall\xi\in X : \pi^+(\xi) = \arg\min_{\mu\in U}\sum_{x^+\in X} P(x^+\mid\xi,\mu)(\mathcal{L}(\xi,\mu) + \gamma v^\pi(x^+)).$$
(Note that there is a slight abuse of notation here since $\pi^+(\xi)\in\Delta(U)$ not $\pi^+(\xi)\in U$; the equation should be interpreted as specifying a deterministic policy.)

In [27]:
def policy_improvement(X,U,P,L,g,v):
    n = len(X); m = len(U)
    pi = np.zeros((n,m))
    for xi in X:
        u = np.argmin([np.sum([P[xi,mu,x]*(L[xi,mu,x] + g*v[x]) for x in X]) for mu in U])
        #assert len(u) == 1
        pi[xi,u] = 1.
    return pi

In [30]:
print('improved policy:')
print(policy_improvement(X,U,P,L,g,policy_evaluation(X,U,P,L,g,pi)))

improved policy:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


Iteratively evaluating and improving the policy defines a *policy iteration* algorithm that converges to the optimal deterministic policy:

In [36]:
def policy_iteration(X,U,P,L,g,pi0):
    pi_ = [pi0]
    v_  = []
    while len(v_) < 2 or not(np.allclose(v_[-1],v_[-2])):
        pi = pi_[-1]
        v  = policy_evaluation(X,U,P,L,g,pi)
        _pi = policy_improvement(X,U,P,L,g,v)
        pi_.append(_pi)
        v_.append(v)
    return pi_,v_

pi_PI,v_PI = policy_iteration(X,U,P,L,g,pi)
print('%d iterations' % len(v_PI))
print('optimal policy:')
print(pi_PI[-1])
print('optimal value:')
print(np.asarray(v_PI[-1]))

3 iterations
optimal policy:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
optimal value:
[12.99380609 13.30836604 13.23951656 13.42796287 13.30285835]


Rather than evaluate the value of the policy, we can iterate toward the optimal policy and value simultaneously:
$$\forall\xi\in X : v_{j+1}(\xi) = \sum_{\mu\in U}\pi_j(\mu\mid\xi)\sum_{x^+\in X} P(x^+\mid\xi,\mu)(\mathcal{L}(\xi,\mu) + \gamma\, v_j(x^+)).$$
$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U}\sum_{x^+\in X} P(x^+\mid\xi,\mu)(\mathcal{L}(\xi,\mu) + \gamma\, v_{j+1}(x^+)].$$

In [32]:
def value_update(X,U,P,R,g,pi,V):
    return np.asarray([np.sum([[pi[xi,mu] * P[xi,mu,x] * (R[xi,mu,x] + g * V[x]) 
                                for x in X] for mu in U]) for xi in X])

In [37]:
def value_iteration(X,U,P,L,g,pi0,maxiter=100):
    n = len(X)
    pi_ = [pi0]
    v_  = [np.zeros(n)]
    while len(v_) < 2 or (not(np.allclose(v_[-1],v_[-2])) and len(v_) < maxiter):
        pi = pi_[-1]
        v  = v_[-1]
        _v = value_update(X,U,P,L,g,pi,v)
        _pi = policy_improvement(X,U,P,R,g,_v)
        pi_.append(_pi)
        v_.append(_v)
    return pi_,v_

pi_VI,v_VI = value_iteration(X,U,P,L,g,pi,200)
print('%d iterations' % len(v_VI))
print('optimal policy:')
print(pi_VI[-1])
print('optimal value:')
print(np.asarray(v_VI[-1]))

171 iterations
optimal policy:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
optimal value:
[14.33347547 14.57269701 14.56239312 14.65294452 14.79329608]


In [35]:
print('policy discrepancy between VI and PI = %d'%np.max(np.abs(pi_VI[-1] - pi_PI[-1])))
print('value discrepancy between VI and PI = %0.1e'%np.max(np.abs(v_VI[-1] - v_PI[-1])))

policy discrepancy between VI and PI = 1
value discrepancy between VI and PI = 1.5e+00


Anticipating future methods that learn optimal policies without access to a system model, we formulate policy and value iteration using the *quality* function (also called *action-value* function) $Q:X\times U\rightarrow\mathbb{R}$ defined by:
$$\forall \xi\in X, \mu\in U : Q^\pi(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma V^\pi(x)].$$
If $\pi$ is optimal, then $Q^\pi$ is related to $V^\pi$ and $\pi$ via:
$$\forall \xi\in X: V^\pi(\xi) = \min_{\mu\in U} Q^\pi(\xi,\mu),\ \pi(\xi) = \arg\min_{\mu\in U} Q^\pi(\xi,\mu).$$

### Q function policy iteration (11.3-43,44 in LewisVrabieSyrmos2008)
$$\forall\xi\in X,\mu\in U : Q_j(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)[R(\xi,\mu,x) + \gamma Q_j(x,\pi_j(x))].$$

$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U} Q_j(\xi,\mu).$$

**This one is more annoying to implement, so I skipped it; I also can't actually make sense of the formula for a nondeterministic policy, due to the \pi_j(x) dependence in the right-hand-side.**

**Here's a proposed alternative formulation that seems to make sense for nondeterministic policies, but the corresponding code doesn't seem to work...**

$$\forall\xi\in X,\mu\in U : Q_j(\xi,\mu) = \sum_{x\in X} P(\xi,\mu)(x)\left[R(\xi,\mu,x) + \gamma \sum_{u\in U}\pi_j(x,u) Q_j(x,u)\right].$$

$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U} Q_j(\xi,\mu).$$

### quality function value iteration (11.3-45,46 in LewisVrabieSyrmos2008)
Just like with *policy iteration* above, iteratively evaluating and improving the policy using the $q$ function defines a *policy iteration* algorithm:
$$\forall\xi\in X,\mu\in U : q_{j+1}(\xi,\mu) = \sum_{x^+\in X} P(x^+\mid\xi,\mu)(\mathcal{L}(\xi,\mu) + \gamma\, q_j(x,\pi_j(x)).$$

$$\forall\xi\in X : \pi_{j+1}(\xi) = \arg\min_{\mu\in U} q_{j+1}(\xi,\mu).$$

In [38]:
def value_update_q(X,U,P,l,g,pi,q):
    """only correct for deterministic policies"""
    _pi = [np.argmax(pix) for pix in pi]
    return np.asarray([[np.sum([P[xi,mu,x] * (L[xi,mu,x] + g * q[x,_pi[x]]) 
                                for x in X]) for mu in U] for xi in X])

In [39]:
_pi = pi_VI[-1]

print ('policy value:')
print (policy_evaluation(X,U,P,L,g,_pi))

q = np.zeros((n,m))
for _ in range(10):
    q = value_update_q(X,U,P,L,g,_pi,q)
print ('policy value from q:')
print (q.min(axis=1))

policy value:
[14.33613249 14.57535403 14.56505014 14.65560154 14.7959531 ]
policy value from q:
[5.52332784 5.76254935 5.75224558 5.84279717 5.74907439]


In [40]:
def policy_improvement_q(X,U,P,L,g,q):
    n = len(X); m = len(U)
    pi = np.zeros((n,m))
    for xi in X:
        u = np.argmin(q[xi])
        pi[xi,u] = 1.
    return pi

In [41]:
def value_iteration_q(X,U,P,L,g,pi0,maxiter=100):
    n = len(X); m = len(U)
    pi_ = [pi0]
    q_  = [np.zeros((n,m))]
    while len(q_) < 2 or (not(np.allclose(q_[-1],q_[-2])) and len(q_) < maxiter):
        pi = pi_[-1]
        q  = q_[-1]
        _q = value_update_q(X,U,P,L,g,pi,q)
        _pi = policy_improvement_q(X,U,P,L,g,_q)
        pi_.append(_pi)
        q_.append(_q)
    return pi_,q_

pi_VI_q,q_VI = value_iteration_q(X,U,P,L,g,pi,200)
print ('%d iterations with q' % len(q_VI))
print ('optimal policy from q:')
print (pi_VI[-1])
print ('optimal value from q:')
print (q_VI[-1].min(axis=1))

print ('policy discrepancy between VI and VI with q = %d'%np.max(np.abs(pi_VI[-1] - pi_VI_q[-1])))
print ('value discrepancy between VI and VI with q = %0.1e'%np.max(np.abs(v_VI[-1] - q_VI[-1].min(axis=1))))

171 iterations with q
optimal policy from q:
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
optimal value from q:
[12.99134795 13.3059079  13.23705842 13.42550472 13.30040021]
policy discrepancy between VI and VI with q = 1
value discrepancy between VI and VI with q = 1.5e+00


### Fig 5.1 in Sutton Barto 1998

When the system model (i.e. $P$ and $R$) is not known but trajectory episodes (i.e. $\{(\xi_t^e,\mu_t^e,x_t^e,r_t^e)\mid t\in T,e\in E\}$) are available, can evaluate the policy using "Monte Carlo" methods -- simply compute the average observed discounted future reward for each state.

**Note:** it's important to only include discounted future reward from the first occurence of each state in each simulation trajectory.  (Not sure if violating this slows or prevents convergence . . .)

In [42]:
def sim(X,U,P,R,x0,pi,T=1):
    n = len(X); m = len(U)
    trj = [[None,None,x0,0.]]
    for t in range(T):
        xi = trj[-1][2]
        mu = np.random.choice(U,p=pi[xi])
        x  = np.random.choice(X,p=P[xi,mu])
        r  = R[xi,mu,x]
        trj.append([xi,mu,x,r])
    return trj[1:]

In [50]:
def mc_policy_evaluation(X,trjs,g):
    n = len(X)
    V = np.zeros(n)
    N = np.zeros(n)
    for trj in trjs:
        x_ = []
        T = len(trj)
        r_ = np.asarray(trj)[:,-1]
        g_ = g**np.asarray(range(T))
        for t,(xi,mu,x,r) in enumerate(trj):
            if xi not in x_:
                V[xi] += np.sum(r_[t:]*g_[:T-t])
                N[xi] += 1
                x_.append(xi)
    return V / N

def mc_policy_evaluation_q(X,trjs,g):
    n = len(X); m = len(U)
    q = np.zeros((n,m))
    N = np.zeros((n,m))
    for trj in trjs:
        xu_ = []
        T = len(trj)
        r_ = np.asarray(trj)[:,-1]
        g_ = g**np.asarray(range(T))
        for t,(xi,mu,x,r) in enumerate(trj):
            if (xi,mu) not in xu_:
                q[xi,mu] += np.sum(r_[t:]*g_[:T-t])
                N[xi,mu] += 1
                xu_.append((xi,mu))
    return q / (N+1)

In [48]:
E = 2000 # number of episodes
T = 20 # horizon of episodes
trjs = [sim(X,U,P,L,np.random.choice(X),_pi,T=T) for e in range(E)]

In [51]:
print ('value of policy:')
print (policy_evaluation(X,U,P,R,g,_pi))
print ('mc value of policy:')
print (mc_policy_evaluation(X,trjs,g))
print ('mc value of policy with q:')
print (mc_policy_evaluation_q(X,trjs,g).max(axis=1)) # hack b/c policy is deterministic
print ('q:')
print (q)
print ('mc q:')
print (mc_policy_evaluation_q(X,trjs,g))

value of policy:
[15.18830653 15.32233441 15.58651968 16.29293235 15.15343541]
mc value of policy:
[7.85385794 7.86677385 7.91296371 8.19586907 8.76430418]
mc value of policy with q:
[7.84989135 7.86276428 7.90894289 8.191736   8.75992422]
q:
[[5.52332784 6.08842241]
 [5.76254935 5.91891724]
 [5.75224558 7.10828245]
 [5.84279717 6.03036829]
 [5.74907439 5.98314866]]
mc q:
[[7.84989135 0.        ]
 [7.86276428 0.        ]
 [7.90894289 0.        ]
 [8.191736   0.        ]
 [0.         8.75992422]]


For a nondeterministic policy, using the same number of samples as above is clearly insufficient:

In [52]:
E = 2000 # number of episodes
T = 20 # horizon of episodes
trjs = [sim(X,U,P,L,np.random.choice(X),pi,T=T) for e in range(E)]

In [53]:
print ('value of policy:')
print (policy_evaluation(X,U,P,L,g,pi))
print ('mc value of policy:')
print (mc_policy_evaluation(X,trjs,g))
print ('mc value of policy with Q:')
#print mc_policy_evaluation_Q(X,trjs,g).min(axis=1)
print (np.sum(mc_policy_evaluation_Q(X,trjs,g)*pi,axis=1))

value of policy:
[19.84294402 19.6297296  20.36225594 19.83700439 19.74344903]
mc value of policy:
[10.63093643 10.50900911 10.85459998 11.19996931 11.1746593 ]
mc value of policy with Q:
[10.39445666  9.36022822  9.71960217 11.15216371 10.34870746]


**I'm currently unsure whether I'm using insufficient number or horizon of episodes, have a bug in my code, or this scheme simply doesn't converge for nondeterminstic policies.**